In [5]:
import pandas as pd 
import numpy as np
import torch
from torch import nn

import torch.nn.functional as F
from torchvision import datasets, transforms

from google.colab import files
uploaded = files.upload()




Saving sample_submission.csv to sample_submission.csv
Saving submission.csv to submission.csv
Saving test.csv to test.csv
Saving train.csv to train.csv


In [22]:
import io
print("I am here!")

my_data = pd.read_csv("train.csv", index_col = 0)

my_data = my_data.drop(columns = ['batch_size_test', 'batch_size_val', 'criterion', 'optimizer', 'batch_size_train'])
y_train = my_data['train_error']
y_val = my_data['val_error']
my_data = my_data.drop(columns = 'train_error')
my_data = my_data.drop(columns = 'val_error')

x = my_data.drop(columns = ['train_loss','val_loss','arch_and_hp','init_params_mu', 'init_params_std', 'init_params_l2'])

col_train = [col for col in x.filter(regex='^train',axis=1).columns]
col_val = [col for col in x.filter(regex='^val',axis=1).columns]
#print(col_train)
#print(col_val)
#print(x)
'''

# ----------------- this part we split the train set to improve the performence of the model ------------

#1. try on training dataset
A = x.drop(['id'],axis=1)
train = A[col_train]
#val = x[col_val]
val = A[col_val]

#2. split 70%/30%

# ==== 70% as the train data ==== y means target ====

split_train_train = train[:int(len(train)*0.7)] #/
split_train_val = val[:int(len(train)*0.7)]

split_y_train_train = y_train[:int(len(train)*0.7)] #/
split_y_train_val = y_val[:int(len(train)*0.7)]


# ==== 30% as the train data ==== y means target ====


split_test_train = train[int(len(train)*0.7):] #/
split_test_val = val[int(len(train)*0.7):]

split_y_test_train = y_train[int(len(train)*0.7):]
split_y_test_val = y_val[int(len(train)*0.7):]

#print(split_train_train)

# === train part (70%) ===

torch_tensor_train = torch.tensor([split_train_train[title].values for title in col_train],dtype=torch.float)
torch_tensor_val = torch.tensor([split_train_val[title].values for title in col_val],dtype=torch.float)

torch_tensor_train = torch.t(torch_tensor_train)
torch_tensor_val = torch.t(torch_tensor_val)

# === test part (30%) ===

torch_tensor_test_train = torch.tensor([split_test_train[title].values for title in col_train],dtype=torch.float)
torch_tensor_test_val = torch.tensor([split_test_val[title].values for title in col_val],dtype=torch.float)

torch_tensor_test_train = torch.t(torch_tensor_test_train)
torch_tensor_test_val = torch.t(torch_tensor_test_val)

'''





I am here!


"\n\n# ----------------- this part we split the train set to improve the performence of the model ------------\n\n#1. try on training dataset\nA = x.drop(['id'],axis=1)\ntrain = A[col_train]\n#val = x[col_val]\nval = A[col_val]\n\n#2. split 70%/30%\n\n# ==== 70% as the train data ==== y means target ====\n\nsplit_train_train = train[:int(len(train)*0.7)] #/\nsplit_train_val = val[:int(len(train)*0.7)]\n\nsplit_y_train_train = y_train[:int(len(train)*0.7)] #/\nsplit_y_train_val = y_val[:int(len(train)*0.7)]\n\n\n# ==== 30% as the train data ==== y means target ====\n\n\nsplit_test_train = train[int(len(train)*0.7):] #/\nsplit_test_val = val[int(len(train)*0.7):]\n\nsplit_y_test_train = y_train[int(len(train)*0.7):]\nsplit_y_test_val = y_val[int(len(train)*0.7):]\n\n#print(split_train_train)\n\n# === train part (70%) ===\n\ntorch_tensor_train = torch.tensor([split_train_train[title].values for title in col_train],dtype=torch.float)\ntorch_tensor_val = torch.tensor([split_train_val[title]

In [23]:
# -------------- the following represents the whole test data set ------------------------

torch_tensor_train = torch.tensor([x[title].values for title in col_train],dtype=torch.float)
torch_tensor_val = torch.tensor([x[title].values for title in col_val],dtype=torch.float)


torch_tensor_train = torch.t(torch_tensor_train)
torch_tensor_val = torch.t(torch_tensor_val)


print(len(col_train))
print(torch_tensor_train.size())


# ------------- test part -----------------------

test_data = pd.read_csv("test.csv", index_col = 0)
test_data = test_data.drop(columns = ['batch_size_test', 'batch_size_val', 'criterion', 'optimizer', 'batch_size_train'])
test_data = test_data.drop(columns = ['arch_and_hp','init_params_mu', 'init_params_std', 'init_params_l2'])
test_train = test_data[col_train]
test_val = test_data[col_val]

torch_tensor_test_train = torch.tensor([test_train[title].values for title in col_train],dtype=torch.float)
torch_tensor_test_val = torch.tensor([test_val[title].values for title in col_val],dtype=torch.float)

torch_tensor_test_train = torch.t(torch_tensor_test_train)
torch_tensor_test_val = torch.t(torch_tensor_test_val)

#print(test_train)
print(torch_tensor_test_train.size())




100
torch.Size([1878, 100])
torch.Size([476, 100])


In [24]:
#we first select a naive model after sorting based on the lowest val_error

#model = torch.nn.Sequential(nn.BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True), nn.Conv2d(3, 32, kernel_size=(1, 1), stride=(1, 1)) , nn.Tanh() , nn.Dropout2d(p=0.4569261378573387, inplace=False) , nn.Conv2d(32, 27, kernel_size=(3, 3), stride=(1, 1)) , nn.ReLU() , nn.Dropout2d(p=0.022897457910152297, inplace=False) , nn.Conv2d(27, 28, kernel_size=(5, 5), stride=(1, 1)) , nn.LeakyReLU(negative_slope=0.5719652455913541) , nn.MaxPool2d(kernel_size=5, stride=5, padding=0, dilation=1, ceil_mode=False) , nn.Flatten() , nn.Linear(in_features=700, out_features=28, bias=True) , nn.LeakyReLU(negative_slope=0.48187737007312437) , nn.Dropout(p=0.3735548245474202, inplace=False) , nn.Linear(in_features=28, out_features=10, bias=True) , nn.BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True) )


input_size = 100
hidden_sizes = [60, 30]
output_size = 1


model_train = nn.Sequential(
                      nn.Linear(input_size, hidden_sizes[0]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[0], hidden_sizes[1]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[1], output_size),
                      )
#nn.Softmax(dim=0)
# Define the loss
criterion = nn.MSELoss()
# Optimizers require the parameters to optimize and a learning rate
optimizer = torch.optim.SGD(model_train.parameters(), lr=0.003)
epochs = 1000
for e in range(epochs):
    running_loss = 0
    for i in range(torch_tensor_train.size()[0]):
        # Training pass
        optimizer.zero_grad()
        
        output = model_train(torch_tensor_train[i])
        #print(output)
        #torch.tensor(y_train[i])
        loss = criterion(output, torch.tensor(y_train[i]))
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

        
model_val = nn.Sequential(
                      nn.Linear(input_size, hidden_sizes[0]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[0], hidden_sizes[1]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[1], output_size),
                      )

# Optimizers require the parameters to optimize and a learning rate
optimizer = torch.optim.SGD(model_val.parameters(), lr=0.008)
epochs = 1000
for e in range(epochs):
    running_loss = 0
    for i in range(torch_tensor_train.size()[0]):
        # Training pass
        optimizer.zero_grad()
        
        output = model_val(torch_tensor_val[i])
        #print(output)
        loss = criterion(output, torch.tensor(y_val[i]))
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

print("finished")



/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:431: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


finished


In [25]:

#now we do the prediction

print("I am here!")
res_train,res_val = [],[]


for row in torch_tensor_test_train:
    res_train.append(model_train(row).item())

for row in torch_tensor_test_val:
    res_val.append(model_val(row).item())

#print(len(res_val))
#print(res_train)

I am here!


In [21]:
'''

preds_train = np.array(res_train)
preds_val = np.array(res_val)


R_square = 1 - (sum((preds_train - split_y_test_train)**2)/sum((preds_train - preds_train.mean())**2))
print(R_square)

R_square = 1 - (sum((preds_val - split_y_test_val)**2)/sum((preds_val - preds_val.mean())**2))
print(R_square)

'''

0.9692663366213935
0.8349412415070974


In [27]:

submit = pd.read_csv("sample_submission.csv", index_col = 0)
res_array = np.zeros(len(res_train)*2)


i = 0
j = 0
while True:
    
    if i == len(res_train):
        break
    
    res_array[j] = res_val[i]
    j += 1
    res_array[j] = res_train[i]
    j += 1
    
    i += 1 

print(res_array)



submit.Predicted = res_array
submit.to_csv("submission.csv")


[ 0.54883868  0.85082996  0.39386749  0.37556946  0.52822775  0.61002874
  0.67002648  0.84686571  0.58538717  0.83720905  0.55757582  0.53027081
  0.73469955  0.88511312  0.4242301   0.38845819  0.67616898  0.80692828
  0.31501901  0.21129754  0.46801496  0.42407858  0.3870149   0.38994023
  0.36243737 -0.02285412  0.49161145  0.63368309  0.41775081  0.22485672
  0.51769352  0.47489434  0.44534716  0.2298882   0.42675769  0.45466068
  0.39131746  0.17932351  0.44834331  0.07845327  0.38032994  0.01696175
  0.41024628  0.51740026  0.60148591  0.77381158  0.29269931  0.01002058
  0.5665884   0.64899039  0.46934178  0.53119862  0.67269188  0.82407022
  0.37272978  0.0422889   0.43653747  0.04758126  0.35215116  0.30813476
  0.76950014  0.87740248  0.37408939  0.22225241  0.70097923  0.79272985
  0.47123185  0.06158581  0.48159188  0.04142225  0.62132889  0.73621362
  0.51696885  0.5578897   0.60754561  0.80464244  0.45649755  0.58189762
  0.45888162  0.1497747   0.65624505  0.60727739  0

In [0]:
files.download('submission.csv')